In [ ]:
"""random_forest.ipynb
by: Archie Gertsman (arkadiy2@illinois.edu)
Project director: Richard Sowers
r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/
Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license
"""

In [4]:
import sys
sys.path.append('../Data/')
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import KFold

In [11]:
df = pd.concat([pd.read_pickle('../Data/block4_29_{}_feat.pkl'.format(i)) for i in [800,830,930]])

In [12]:
df_agg = df[np.isin(df['type'], ['Car','Taxi'])] \
    .groupby(['id','traj']).agg({
        'xtrack_dist': 'std',
        'avg_surr_speed': 'mean',
        'avg_surr_speed': 'std',
        'lon_acc': 'std',
        'lat_acc': 'std',
        'type': 'first'
    }) \
    .reset_index() \
    .drop(['id','traj'], axis=1)

len(df_agg[df_agg['type'] == 'Car']), len(df_agg[df_agg['type'] == 'Taxi'])

(1415, 493)

In [29]:
n_taxi_copies = 2
taxi = df_agg[df_agg['type']=='Taxi']
taxi = pd.concat([taxi for i in range(n_taxi_copies)])
df2 = pd.concat([df_agg, taxi])

g = df2.groupby('type')
df2 = g.apply(lambda x: x.sample(g.size().min())).reset_index(drop=True)

k = 10
kf = KFold(n_splits=k, shuffle=True)
accs = np.zeros(k)
X,y = df2.drop('type', axis=1), df2['type']

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = AdaBoostClassifier()
    model.fit(X_train, y_train)

    y_hat = model.predict(X_test)
    a = y_hat==y_test
    accs[i] = len(a[a==True]) / len(y_test)

print('{}-fold model accuracy'.format(k))
print('mean: {}%'.format(round(100*accs.mean(), 3)))
print('std: {}%'.format(round(100*accs.std(), 3)))

10-fold model accuracy
mean: 69.258%
std: 2.12%
